In [42]:
from dataset import TranSiGenDataset
from model import TranSiGen
from utils import *
import pickle
import argparse
import warnings
import torch
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Carregamento dos dados

Os dados são carregados a partir de um arquivo `.h5`:

In [45]:
data_path = '../data/LINCS2020/data_example/processed_data_id.h5'

In [46]:
data = load_from_HDF(data_path)

O conteúdo do arquivo é um dicionário contendo as seguintes informações:

In [47]:
print(", ".join(data.keys()))

LINCS_index, canonical_smiles, cid, sig


- **LINCS_index**: 
- **canonical_smiles**: ids das moléculas dos compostos químicos. Esse id é convertido para o padrão Smiles usando o vetor `idx2smi` definido posteriormente.
- **cid**: cell line identifier
- **sig**: signature

Esse conjunto é, em resumo, um ponteiro para os dados. Os dados em si serão carregados mais à frente pela classe `TranSiGenDataset`.

## Definição das configurações gerais
Observações:
- `cell_count`: número de linhagens celulares. Usado apenas para identificar o modelo salvo.
- `feat_type`: tipo de representação das moléculas. Para o uso no modelo, o código Smiles é convertido para uma outra representação da molécula. Essa conversão pode ser feita tanto para um embedding pelo modelo pré-treinado KPGT, ou pela impressão digital molecular (*molecular fingerprint*) ECFP4.
- `split_type`: define como os dados serão dividos em treino, validação e teste. Os possíveis valores são
    - `random_split`: essa divisão é feita de forma aleatória.
    - `cell_split`: os conjuntos são dividos considerando uma mesma linhagem celular, para que não haja o risco de todos os dados de uma célula (ou boa parte deles) fiquem apenas em um dos conjuntos.
- `features_dim`: tamanho do vetor que representa a molécula. No caso do KPGT, esse valor é 2304. No caso do ECFP4, o valor é 2048.

In [48]:
cell_count = len(set(data['cid']))
feat_type = 'KPGT'
batch_size = 64
learning_rate = 1e-3
beta = 0.1
dropout = 0.1
weight_decay = 1e-5
n_folds = 5
random_seed = 364039
split_type = 'smiles_split'
features_dim = 2304
features_embed_dim = [400]
n_latent = 100
init_mode = 'pretrain_shRNA'
# init_mode = 'random'
n_epochs = 300
molecule_path = '../data/LINCS2020/idx2smi.pickle'

In [49]:
local_out = '../results/trained_models_{}_cell_{}/{}/feature_{}_init_{}/'.format(cell_count, split_type, random_seed, feat_type, init_mode)

Abaixo é carregador o vetor `idx2smi`, responsável por converter os índices das moléculas carregadas acima nos respectivos códigos Smiles.

In [50]:
with open(molecule_path, 'rb') as f:
    idx2smi = pickle.load(f)

Exemplo:

In [51]:
idx2smi[2]

np.str_('BrCC(=O)NCCc1ccc2ccccc2c1')

## Divisão de treino e teste

In [52]:
pair, pairv, pairt = split_data(data, n_folds=n_folds, split_type=split_type, rnds=random_seed)

In [53]:
train = TranSiGenDataset(
    LINCS_index=pair['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pair['canonical_smiles'],
    cid=pair['cid']
)

valid = TranSiGenDataset(
    LINCS_index=pairv['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pairv['canonical_smiles'],
    cid=pairv['cid']
)

test = TranSiGenDataset(
    LINCS_index=pairt['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pairt['canonical_smiles'],
    cid=pairt['cid']
)

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)
valid_loader = torch.utils.data.DataLoader(dataset=valid, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)

## Criação do Modelo

In [54]:
model = TranSiGen(
    n_genes=978,
    n_latent=n_latent,
    n_en_hidden=[1200],
    n_de_hidden=[800],
    features_dim=features_dim,
    features_embed_dim=features_embed_dim,
    init_w=True,
    beta=beta,
    device=dev,
    dropout=dropout,
    path_model=local_out,
    random_seed=random_seed
)

In [55]:
_ = model.to(dev)

### Arquitetura do Modelo
Arquitetura dos codificadores do $X_1$ e do $X_2$ (são iguais):

In [56]:
model.encoder_x1

Sequential(
  (0): Linear(in_features=978, out_features=1200, bias=True)
  (1): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
)

Arquitetura dos decodificadores do $X_1$ e do $X_2$ (também são iguais):

In [57]:
model.decoder_x2

Sequential(
  (0): Linear(in_features=100, out_features=800, bias=True)
  (1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
  (4): Linear(in_features=800, out_features=978, bias=True)
  (5): ReLU()
)

Arquitetura do embedder da molécula:

In [58]:
model.feat_embeddings

Sequential(
  (0): Linear(in_features=2304, out_features=400, bias=True)
  (1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
)

## Treinamento/Carregamento do Modelo

In [59]:
if init_mode == 'pretrain_shRNA':
    # Carregando modelo pré-treinado
    print('=====load vae for x1 and x2=======')
    model_dict = model.state_dict()
    filename = '../results/trained_model_shRNA_vae_x1/best_model.pt'
    model_base_x1 = torch.load(filename, map_location='cpu')
    model_base_x1_dict = model_base_x1.state_dict()
    for k in model_dict.keys():
        if k in model_base_x1_dict.keys():
            model_dict[k] = model_base_x1_dict[k]
    filename = '../results/trained_model_shRNA_vae_x2/best_model.pt'
    model_base_x2 = torch.load(filename, map_location='cpu')
    model_base_x2_dict = model_base_x2.state_dict()
    for k in model_dict.keys():
        if k in model_base_x2_dict.keys():
            model_dict[k] = model_base_x2_dict[k]
    model.load_state_dict(model_dict)
    del model_base_x1, model_base_x2
else:
    epoch_hist, best_epoch = model.train_model(
        train_loader=train_loader,
        test_loader=valid_loader,
        n_epochs=n_epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        save_model=False
    )

=====load vae for x1 and x2=======


## Avaliação do Modelo no Conjunto Teste

In [60]:
_, _, test_metrics_dict_ls = model.test_model(loader=test_loader, metrics_func=['pearson'])

for name, rec_dict_value in zip(['test'], [test_metrics_dict_ls]):
    df_rec = pd.DataFrame.from_dict(rec_dict_value)
    smi_ls = []
    for smi_id in df_rec['cp_id']:
        smi_ls.append(idx2smi[smi_id])
    df_rec['canonical_smiles'] = smi_ls

In [61]:
df_rec

,x1_rec_pearson,x2_rec_pearson,x2_pred_pearson,DEG_rec_pearson,DEG_pred_pearson,cp_id,cid,sig,canonical_smiles
0,0.977669,0.964104,0.803589,0.561124,0.221309,9,A549,A549_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
1,0.967655,0.981851,0.848939,0.552822,0.168256,6,PC3,PC3_6,Brc1c[nH]c2nc(SCc3ccccc3C#N)nc2c1
2,0.975305,0.974306,0.628056,0.688416,0.252797,9,A375,A375_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
3,0.957094,0.967181,0.772312,0.687505,0.244974,9,MCF7,MCF7_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
4,0.974583,0.982083,0.844031,0.468299,0.166315,6,HT29,HT29_6,Brc1c[nH]c2nc(SCc3ccccc3C#N)nc2c1
5,0.966004,0.966937,0.839300,0.614072,0.184055,9,PC3,PC3_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
6,0.954278,0.977408,0.829878,0.602793,0.253246,13,HA1E,HA1E_13,Brc1ccc(NC(=O)N2NC(=O)[C@H]([C@@H]2c2ccccc2)c2ccccc2)cc1
7,0.961352,0.975592,0.650044,0.577168,0.271229,9,HA1E,HA1E_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
8,0.973674,0.966812,0.851971,0.431136,0.261810,9,HT29,HT29_9,Brc1ccc(CSc2nnc(c3ccccn3)n2Cc4ccco4)cc1
9,0.963402,0.973458,0.799977,0.419056,0.168240,6,HA1E,HA1E_6,Brc1c[nH]c2nc(SCc3ccccc3C#N)nc2c1
